To work this needs the 
- Secondary structure of aligned tRNAs.
- Aligned tRNAs
- User specified sections (for now)

Current issues:
- I cut corners nad the loops regex picks up all the arms. Becasue of that if there are weird bubbles, they are skipped over I should fix this, but also I am a bit lazy atm
- 

In [1]:
import re

# Incredibly vital regular expressions that <br>pick up the different portions of the tRNA

- startAA
- endAA
- loops
- introns

In [131]:
example_structures = ['(((((((,,<<<<___.___.__>>>>,<<<<<_______>>>>>,,...........,,,<<<<<_______>>>>>))))))):',
                      '(((((((,,<<<<___.___.__>>>>,<<<<<______................................................._>>>>>,,..........,,,<<<<<_______>>>>>))))))):'
                        ]
startAA = re.compile(r'\(+[^<>]{0,}')#r'\(+.{0,}?<'

endAA = re.compile(r'\)+:')#r'.{1}\)+'

loops = re.compile(r'<+.{3,}?>+')

introns = re.compile(r'[^<>]*\.{3,}[^<>]*')


patterns = [startAA,
            endAA,
            loops,
            introns
            
            ]



# The dictionary of lists that is populated by the spans

## Intended use
User says they want D-arm and T-arm regions.
The code loops through turning each region
NNN, unless it was specified then it leaves
the sequence alone.

## Example
method('se--quence', 'T-arm', 'D-arm')  

'se--quence'  

'NN--quence'

'NN--quNNce'

In [31]:
alignedRegions = {  'AA-stem': [],
                    'D-arm': [],
                    'Anticodon loop': [],
                    'Introns': [],
                    'T-arm': []
                 }

## Actually populates the dictionary

Note: will need to clear the dictionary every population cycle
because otherwise it will keep adding to the same dictionary
over and over again.  

The solution can be as simple as just re-initializing it


In [32]:
for n, pattern in enumerate(patterns):
    #loops through the different regular expression patterns
    matches = pattern.finditer(example_structures[1])
    
    for matchNum, match in enumerate(matches):
        #loops through each match to the pattern
        if n==0 or n==1:
            alignedRegions['AA-stem'].append( match.span() )
        elif n==2:
            if matchNum==0:
                alignedRegions['D-arm'].append( match.span() )
            elif matchNum==1:
                alignedRegions['Anticodon loop'].append( match.span() )
            else:
                alignedRegions['T-arm'].append( match.span() )
        else:
            alignedRegions['Introns'].append( match.span() )
        
            


In [33]:
print(alignedRegions)

{'AA-stem': [(0, 9), (126, 134)], 'D-arm': [(9, 27)], 'Anticodon loop': [(28, 94)], 'Introns': [(33, 89), (94, 109)], 'T-arm': [(109, 126)]}


In [34]:
def properShape(ar):
    '''Returns True if the dictionary has the proper shape of a tRNA'''
    a = len( ar['AA-stem'] ) == 2
    b = len( ar['D-arm'] ) == 1
    c = len( ar['Anticodon loop'] ) == 1
    d = len( ar['T-arm'] ) == 1
    return a and b and c and d

properShape(alignedRegions)

True

# Puts the dictionary into a list

In [35]:
orderRegion = []
for key, value in alignedRegions.items():
    for rang in value:
        orderRegion.append(rang)#example_structures[0][rang[0] : rang[1]]
        
print(sorted(orderRegion))
orderRegion.sort()

[(0, 9), (9, 27), (28, 94), (33, 89), (94, 109), (109, 126), (126, 134)]


# Makes the list into the SS

In [39]:
previous=(-1,-1)
for rang in orderRegion:
    if previous and rang[0] not in range(previous[0], previous[1]):
        print( example_structures[1][rang[0] : rang[1]] , end='' )
    previous = rang
    
print('\n', example_structures[1])

(((((((,,<<<<___.___.__>>>><<<<<______................................................._>>>>>,,..........,,,<<<<<_______>>>>>))))))):
 (((((((,,<<<<___.___.__>>>>,<<<<<______................................................._>>>>>,,..........,,,<<<<<_______>>>>>))))))):


In [178]:
class CreateWindows:
    '''Given a single sequence this will convert it into just the regions we are interested in'''
    import re
    startAA = re.compile(r'\(+[^<>]{0,}')#r'\(+.{0,}?<'
    endAA = re.compile(r'\)+:')#r'.{1}\)+'
    loops = re.compile(r'<+.{3,}?>+')
    introns = re.compile(r'[^<>]*\.{3,}[^<>]*')
    patterns = [startAA, endAA, loops, introns]

    
    def __init__(self, secondaryStructure, sequence='', desiredRegions=['AA-stem','D-arm','Anticodon loop', 'T-arm', 'Introns']):
        self.ss = secondaryStructure
        self.alignedRegions = self.populateDictionary()
        self.userInput = sequence
        self.desiredRegions = desiredRegions
        
    def populateDictionary(self):
        patterns = self.patterns
        alignedRegions = {  'AA-stem': [],
                            'D-arm': [],
                            'Anticodon loop': [],
                            'Introns': [],
                            'T-arm': []
                         }
        
        for n, pattern in enumerate(patterns):
            #loops through the different regular expression patterns
            matches = pattern.finditer(self.ss)

            for matchNum, match in enumerate(matches):
                #loops through each match to the pattern
                if n==0 or n==1:
                    alignedRegions['AA-stem'].append( match.span() )
                elif n==2:
                    if matchNum==0:
                        alignedRegions['D-arm'].append( match.span() )
                    elif matchNum==1:
                        alignedRegions['Anticodon loop'].append( match.span() )
                    else:
                        alignedRegions['T-arm'].append( match.span() )
                else:
                    alignedRegions['Introns'].append( match.span() )
        
        return alignedRegions
    
    def isProperShape(self):
        '''Returns True if the dictionary has the proper shape of a tRNA'''
        ar = self.alignedRegions
        a = len( ar['AA-stem'] ) == 2
        b = len( ar['D-arm'] ) == 1
        c = len( ar['Anticodon loop'] ) == 1
        d = len( ar['T-arm'] ) == 1
        
        return a and b and c and d

    def converted2N(self):
        '''Returns a new fasta string, where every character is replaced by N'''
        newSequence = ''
        for n, letter in enumerate(self.userInput):
            if letter.isalpha():
                newSequence += "N"
            else:
                newSequence += "-"
        return newSequence
                
 
    def getDesiredRanges(self):
        '''Returns list of the ranges where we dont want just N'''
        ranges = []
        for rang in self.desiredRegions:
            ranges.extend( self.alignedRegions[rang] )
        return ranges
    
    def getWindowedSequence(self):
        newSequence = self.converted2N()
        for rang in self.getDesiredRanges():
            for position in range(rang[0], rang[1]):
                newSequence = newSequence[:position] + self.userInput[position] + newSequence[position+1:]
        return newSequence

In [192]:
sacc = "GGGCGUGUGGCGUAGUcGGU--AGCGCGCUCCCUUAGCAUGGGAGAG-----------GU\
CUCCGGUUCGAUUCCGGACUCGUCCA"

yarrowia = "GGGCGUCUGGUGUAGUuGGU-UAUCACAUGCGCUUAGCA---------------------\
----------------------------UGCGCGAG----------GUCCCCGGUUCGAU\
UCCGGUGUCGUCCA"

windowmaker = CreateWindows( example_structures[0], sequence=sacc, desiredRegions=['AA-stem, "T-arm"] )
windowmaker2 = CreateWindows( example_structures[1], sequence=yarrowia, desiredRegions=['AA-stem, "T-arm"] )

SyntaxError: EOL while scanning string literal (<ipython-input-192-5a120a6ac90a>, line 8)

input --> convert all to N and dash --> fill in designated regions with letters

In [193]:
windowmaker.getWindowedSequence()

KeyError: 'AA-stem, "T-arm"'

In [194]:
windowmaker2.getWindowedSequence()

KeyError: 'AA-stem, "T-arm"'